In [627]:
include("../MFAscan_ens.jl")
using Random
using LatticeModels
using Plots
plotly()

Plots.PlotlyBackend()

In [628]:
struct CubicParams
    L::Int
    W::Float64
end

In [629]:
function f(p::CubicParams; rng = Random.GLOBAL_RNG)
    H = cubic(L = p.L, M = p.L, N = p.L) .+ p.W*Diagonal(rand(L*L*L) .- 0.5)
end

function g(p::CubicParams; rng = Random.GLOBAL_RNG)
    H = square(M = p.L, N = p.L) .+ p.W*Diagonal(rand(L*L) .- 0.5)
end

function h(p::CubicParams; rng = Random.GLOBAL_RNG) #Aubry Andre
    H = chain(N = p.L) .+ p.W*Diagonal(cos.(2π*(1:L)*(1 + √5)/2 .+ 2*pi*rand(rng)))
end

h (generic function with 1 method)

In [630]:
L = 10; W = 32.5; E_c = 0.; E_dl = 0.3
l = [2]
q = collect(-3:0.3:3)

p = CubicParams(L, W)
ltc = Lattice3D(L, L, L, 1)

Lattice3D{Int64}(10, 10, 10, 1)

In [632]:
E, gipr, μqlnμ, τ, α, f_α = scan_ταf(f, p, E_c, E_del, ltc, L, l=l, q=q, R=1, nev = 100, seed = rand(1:19999))

Hello!


(-0.007914372935846157, [2.4710157318779825e26; 5.0529436058562776e23; … ; 0.31419718001510966; 0.28205134549367267;;], [-5.09584161013999e28 -9.34952260002043e25 … -0.12295187297955569 -0.11375784157682511; -1.3362189418611687e28 -2.861982455724812e25 … -0.12601402669409323 -0.11631676334701273; … ; -3.1343314807218393e25 -1.3952307970333312e23 … -0.12936283092941675 -0.11869219863287708; -4.6526627780492536e24 -2.379134319150216e22 … -0.14055710122099407 -0.12393018500041504;;;], [-37.759668292841674; -33.91210538966236; … ; 0.7193409073959887; 0.786402593489926;;], [12.840710064493244; 12.807950224889751; … ; 0.2354309302244882; 0.21271993858724333;;], [-0.7624619006380584; -0.6693602175399747; … ; -0.08367739578987055; -0.14824277772819605;;])

In [633]:
plot(log.(l./L), log.(gipr'), label = q', line=:dash, marker = :circle)

In [634]:
p1 = plot(q, τ, legend = false)
p2 = plot(q, α, legend = false)
p3 = plot(α, f_α)
p4 = plot(p1, p2, p3)

In [635]:
plot(gipr)